# <font color='darkorchid'>Wilson Dam</font>
<font color = 'slategray' > This notebook provides analyzes simulates the operations at Wilson dam, using the Canteen package. </font>

by: John Kucharski | 12 July 2021


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import interpolate

# <font color = darkviolet> Wilson Reservoir </font>
<font color = slategray> This notebook analyzes the Wilson Reservoir in the xx watershed in Kansas. 

In [2]:
import sys
import numpy as np
from scipy import interpolate

In [3]:
sys.path.insert(0, '/Users/johnkucharski/Documents/source/canteen')
import src.outlet as outlet
import src.reservoir as reservoir
import src.utilities as utilities

## <font color = salmon> Global Variables </font>
<font color = slategray> A few global variables used in this notebook are defined here. </font>

In [4]:
data_folder: str = '/Users/johnkucharski/Documents/data/usace/ks/'

## <font color = salmon> Reservoir </font>

### <font color = slateblue> map: volume-elevation (rating curve) </font>
<font color = slategray> A lot of the Wilson dam and reservoir data is expressed in terms of elevation, instead of volume. As a result, the rating curve needs to be established first. Later on, I plot this data. </font> 

In [5]:
rating_path: str = data_folder + 'wilson_rating_curve.csv'

volumes = np.loadtxt(rating_path, skiprows=1, delimiter=',', usecols=[1])/1000 #taf
elevations = np.loadtxt(rating_path, skiprows=1, delimiter=',', usecols=[0])   #ft

f_rating = interpolate.interp1d(volumes, elevations, fill_value=(elevations[0], elevations[-1]), bounds_error=False)
inversef_rating = interpolate.interp1d(elevations, volumes, fill_value=(volumes[0], volumes[-1]), bounds_error=False)

rating_map = reservoir.Map(name='elevation', f=f_rating, inverse_f=inversef_rating)

### <font color = slateblue> outlets </font>
<font color = slategray> The reservoir contains two outlets:
 - gate: a single release gate
 - spillway: a spillway with no operational control

In [8]:
gate_path = data_folder + 'wilson_outlets.csv'

elevations = np.loadtxt(gate_path, skiprows=1, delimiter=',', usecols=[0]) # ft
gate_max_cfs = np.loadtxt(gate_path, skiprows=1, delimiter=',', usecols=[1]) # cfs
spillway_cfs = np.loadtxt(gate_path, skiprows=1, delimiter=',', usecols=[2]) # cfs

gate_location, spillway_location = 1450.5, 1582.5
volumes = [inversef_rating(x) for x in elevations]
gate_max_taf = [x * utilities.days_to_sec(1) * utilities.cf_to_af(1) * utilities.af_to_taf(1) for x in gate_max_cfs]
spillway_taf = [x * utilities.days_to_sec(1) * utilities.cf_to_af(1) * utilities.af_to_taf(1) for x in spillway_cfs]

gate = outlet.Outlet('gate', gate_location, utilities.f_interpolate_from_data(volumes, gate_max_taf))
spillway = outlet.Outlet('spillway', spillway_location, utilities.f_interpolate_from_data(volumes, spillway_taf))

wilson_outlets = [gate, spillway]

In [9]:
k = inversef_rating(1592)
wilson = reservoir.Reservoir(capacity=k, outlets=wilson_outlets, maps={rating_map})
wilson.print()

TypeError: type numpy.ndarray doesn't define __round__ method

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax.scatter(volumes, elevations, edgecolors='grey', facecolors='none', label='obs')
ax.plot(volumes, rating_fx(volumes), color='red', linestyle='dashed', label='fit')
ax.set_xlabel('volume (TAF)')
ax.set_ylabel('elevation (ft)')
ax.set_title('Wilson Reservoir Rating Curve')
ax.legend()
plt.show()

In [ ]:
k = inversef_rating(1592)
wilson = reservoir.Reservoir(capacity=k, outlets=[gate, ])

elevations = np.loadtxt(rating_path, skiprows=1, delimiter=',', usecols=[0]) # ft
gate = np.loadtxt(rating_path, skiprows=1, delimiter=',', usecols=[1]) # cfs